# Mock project

## Imports

In [ ]:
# Imports
from music21 import note, stream, midi, meter, tempo, pitch, instrument

from typing import Union, List

import pandas as pd
import numpy as np

import mchmm    # Markov models
import copy
from iteration_utilities import deepflatten
from fractions import Fraction

import math
import random

import scipy.io.wavfile as wave
from IPython.display import Audio

## Helper functions

In [ ]:
def play(score: stream.Stream) -> None:
    """Shortcut to play a score

    Args:
        score (stream.Stream): The score to play
    """    
    midi.realtime.StreamPlayer(score).play()

## Base rhythms

In [ ]:
UNIT = 4

def elem_to_note(elem: Union[float, str]) -> Union[note.Note, note.Rest]:
    """Creates a note or rest from a number or textual representation  

    Args:
        elem (Union[float, str]): The element to convert 
            (float for note and str for rest)

    Returns:
        Union[note.Note, note.Rest]: The note or rest generated
    """    
    if isinstance(elem, str):
        return note.Rest(quarterLength = UNIT * Fraction(elem))
    else:
        return note.Note('C', quarterLength = UNIT * elem)

In [ ]:
sequences = [
    [1/4, 1/4, 1/4, 1/4],
    [1/4, 1/4, 1/2],
    [3/16, 1/16, 1/8, 1/8, 1/4, 1/4],
    [1/4, 1/4, 1/2]
]

rhythms = []
for seq in sequences:
    notes_seq = list(map(elem_to_note, seq))
    rhythms.append(notes_seq)

## Markov chain

In [ ]:
_, goat_wav = wave.read('goat.wav')
goat_list = [abs(sum(c)) % len(rhythms) for c in goat_wav if abs(sum(c))<=len(rhythms)*random.randint(0,len(rhythms))]

markov = mchmm.MarkovChain().from_data(goat_list)

# visualization
table = pd.DataFrame(markov.observed_matrix, index = markov.states, columns = markov.states, dtype = int)
table.div(table.sum(axis=1), axis=0)

## Melody

In [ ]:
# SCORE SETUP
def setup(stream: stream.Stream) -> None:
    """Does a stream setup (i.e. put the time signature and the tempo)

    Args:
        stream (stream.Stream): The stream to set up
    """    
    stream.insert(0, meter.TimeSignature('4/4'))
    stream.insert(0, tempo.MetronomeMark('allegretto'))

In [ ]:
# GENERATE THE MELODIES
def gen_melody(markov: mchmm.MarkovChain, rhythms: List[List[Union[note.Note, note.Rest]]],
               length:int = None, start:int = None,
               prev_melody=None, prev_states=None, ratio=1/2):
    #TODO finish doc
    
    melody = stream.Part()
    cont_melody = prev_melody is not None and prev_states is not None

    # adjust parameters if it is a continuing melody
    if cont_melody:
        length = math.floor(len(prev_states) / ratio) - len(prev_states)

        # find the start by generating it from the last state
        # (need item() to convert to python integer)
        start = markov.simulate(2, ret='states', start=prev_states[-1].item())[1].item()

        # put the previous melody's notes
        for n in prev_melody.notesAndRests:
            melody.append(copy.deepcopy(n))


    states = markov.simulate(length, ret='states', start=start)

    # put the generated notes in the melody
    notes_list = [copy.deepcopy(rhythms[i.item()]) for i in states]

    for notes in notes_list:
        for note in notes:
            melody.append(note)

    setup(melody)

    # change speed if the melody is a continuing melody
    if cont_melody:
        melody = melody.augmentOrDiminish(ratio, inPlace=False)

    return melody, states


## Pitches

In [ ]:
# PITCHES
nice_scale = [pitch.Pitch(p) for p in ('Db', 'F#', 'G#', 'B')]

In [ ]:
def put_pitches(part: stream.Stream, pitches_list: List[pitch.Pitch]) -> None:
    """Puts pitches in a stream

    Args:
        part (stream.Stream): The stream
        pitches_list (List[pitch.Pitch]): The pitches
    """    
    for n in part.notes:
        n.pitch = random.choice(pitches_list)

## Timbres

In [ ]:
# TIMBRES
instruments = [instrument.Accordion(), instrument.Violin(), instrument.Clarinet(), instrument.Woodblock()]

## Create melodies

In [ ]:
# TRANSFORM STREAMS
def repeat_until(part: stream.Stream, bars: int) -> None:
    """Repeats the notes in a part until there are a certain number of bars.\n
    The stream should contain a MetronomeMark and a TimeSignature

    Args:
        part (stream.Stream): The stream
        bars (int): The number of bars
    """
    bpm = part.getElementsByClass('MetronomeMark')[0].number
    quarters = part.getElementsByClass('TimeSignature')[0].barDuration.quarterLength

    seconds_per_bar = quarters / bpm * 60

    cur_bars = part.seconds / seconds_per_bar
    notes = list(part.notesAndRests)

    total_notes = bars / cur_bars * len(notes)
    
    for i in range(math.floor(total_notes) - len(notes)):
        i = i % len(notes)
        part.append(copy.deepcopy(notes[i]))


def shift(part: stream.Stream, bars: int) -> None:
    """Shift a stream by a certain number of bars

    Args:
        part (stream.Stream): The stream to shift
        bars (int): The number of bars we need to shift
    """    
    notes = copy.deepcopy(part.notesAndRests)
    for n in part.notesAndRests:
        part.remove(n)

    q_per_bar = part.getElementsByClass('TimeSignature')[0].barDuration.quarterLength
    part.append(note.Rest(quarterLength = q_per_bar * bars))

    for n in notes:
        part.append(n)


def replace_with_rests(part: stream.Stream, nbr_rests: int) -> None:
    """Replaces randomly some notes in a stream with rests

    Args:
        part (stream.Stream): The stream
        nbr_rests (int): The number of rests to add
    """    
    notes = part.notes
    if len(notes) < nbr_rests:
        nbr_rests = len(notes)

    to_replace = random.choices(notes, k=nbr_rests)
    for n in to_replace:
        part.replace(n, note.Rest(quarterLength=n.quarterLength))

    

In [ ]:
# Create melodies
melody0, states0 = gen_melody(markov, rhythms, length=4)
melody1, states1 = gen_melody(markov, rhythms, prev_melody=melody0, prev_states=states0)
melody2, states2 = gen_melody(markov, rhythms, prev_melody=melody1, prev_states=states1)

base_melodies = [melody0, melody1, melody2]
part_melodies = copy.deepcopy(base_melodies)

# Setup base
repeat_until(melody0, 8)

repeat_until(melody1, 6)
shift(melody1, 2)

shift(melody2, 4)

# Extend part melodies
for m in part_melodies:
    repeat_until(m, 8)

# Generate all parts
PARTS = 3
all_parts = [base_melodies]

for i in range(PARTS - 1):
    all_parts.append(copy.deepcopy(part_melodies))

# put all pitches
#FIXME transpose doesn't work
half_steps = 0
rests = 5

for part in all_parts:
    for melody in part:
        put_pitches(melody, nice_scale)
        replace_with_rests(melody, rests)
        #melody.transpose(interval.Interval(half_steps), inPlace=True, recurse=True)

    half_steps += 4
    rests = rests**2


# Flatten to streams
nbr_parts = len(all_parts)
nbr_streams = len(all_parts[0])

streams = []

for i in range(nbr_streams):
    mel = stream.Part()

    for j in range(nbr_parts):
        for n in all_parts[j][i].notesAndRests:
            mel.append(n)

    streams.append(mel)

# add instruments
for s in streams[:-1]:
    s.insert(0, instruments[i % (len(streams) - 1)])

streams[-1].insert(0, instruments[3])

In [ ]:
score = stream.Score()

for s in streams[:-1]:
    score.insert(0, s)

score.insert(0, streams[-1])

setup(score)

print(score.seconds / 60, "minutes")
score.show('lily.pdf')

In [ ]:
score.write('lily.pdf', fp='score')
score.write('midi', fp='score.mid')

In [ ]:
play(score)